In [1]:
import os

In [2]:
list_file = list(os.walk('./menu'))[0][2]
if '.DS_Store' in list_file:
    list_file.remove('.DS_Store')
if '0.null.txt' in list_file:
    list_file.remove('0.null.txt')
    
list_file.sort()
    
len(list_file)

In [3]:
import pandas as pd

In [4]:
df = pd.DataFrame()

df

""


In [5]:
from tqdm import tqdm
from bs4 import BeautifulSoup
from lxml import etree

In [6]:
for file in tqdm(list_file, desc='Progress', ncols=77):
    with open('./menu/' + file, 'r', encoding='UTF-8') as f:
        html = f.read()
        
        # = = = = = = = = = = = = = = =
        
        soup = BeautifulSoup(html, 'lxml')
        
        # = = = = = = = = = = = = = = =
        
        html = etree.HTML(str(soup))
        
        list_part = html.xpath('//a[@class="product-hit-link"]')
        
        # = = = = = = = = = = = = = = =

        list_url = ['https://www.atp-autoteile.de' + part.xpath('./@href')[0].strip() for part in list_part]

        # = = = = = = = = = = = = = = =
        
        list_kit = []
        for part in list_part:
            kit = ''
            list_tr = part.xpath('./descendant::div[@class="product-detail-info"]/table/tbody/tr')
            for tr in list_tr:
                if len(tr.xpath('./@title')) != 0:
                    name = tr.xpath('./@title')[0].strip()
                    amount = tr.xpath('./td/strong/text()')[0].strip().replace(name, '').strip()
                    if kit.endswith('; '):
                        kit = kit[:-2] + '}\n{name: ' + name + '; amount: ' + amount + '; '
                    else:
                        kit += '}\n{name: ' + name + '; amount: ' + amount + '; '
                else:
                    if len(tr.xpath('./td[2]/text()')) == 0:
                        kit += tr.xpath('./td[1]/text()')[0].strip() + '; '
                    else:
                        kit += tr.xpath('./td[1]/text()')[0].strip() + ': ' + tr.xpath('./td[2]/text()')[0].strip() + '; '

            if kit.endswith('; '):
                kit = kit[2:-2] + '}'
            else:
                kit = kit[2:] + '}'

            list_kit.append(kit)
        
        # = = = = = = = = = = = = = = =
        
        df_temp = pd.DataFrame({'Page': int(file.replace('.txt', '')),
                                'No.': [i+1 for i in range(len(list_part))],
                                'Url': list_url,
                                'Kit': list_kit})

        df = pd.concat([df, df_temp], ignore_index=True).fillna('')

# = = = = = = = = = = = = = = =
        
df = df.drop_duplicates(ignore_index=True)
df = df.sort_values(by=['Page', 'No.'], ascending=[True, True]).reset_index(drop=True)
df['No.'] = [i+1 for i in range(len(df))]
df.to_excel('./menu.xlsx', index=False)

df

Progress: 100%|██████████████████████████████| 55/55 [00:12<00:00,  4.32it/s]


,Page,No.,Url,Kit
0,1,1,https://www.atp-autoteile.de/de/product/431665...,{name: Bremsbeläge vorne (Satz); amount: 1x; B...
1,1,2,https://www.atp-autoteile.de/de/product/988398...,{name: Bremssattel vorne links belüftete Brems...
2,1,3,https://www.atp-autoteile.de/de/product/430932...,{name: Verschleißwarnkontakt Hinterachse; amou...
3,1,4,https://www.atp-autoteile.de/de/product/102067...,{name: Bremsbeläge vorne (Satz); amount: 1x; A...
4,1,5,https://www.atp-autoteile.de/de/product/430944...,{name: Verschleisswarnkontakt Hinterachse; amo...
...,...,...,...,...
1353,55,1354,https://www.atp-autoteile.de/de/product/373129...,"nge:: 54,1 mm; Höhe:: 64,6 mm; Dicke/Stärke:: ..."
1354,55,1355,https://www.atp-autoteile.de/de/product/372619...,"eite:: 55,8 mm; Bremssystem:: ATE; Dicke/Stärk..."
1355,55,1356,https://www.atp-autoteile.de/de/product/372777...,eite:: 100 mm; Bremssystem:: Lucas / TRW; Dick...
1356,55,1357,https://www.atp-autoteile.de/de/product/364739...,"eite:: 82,6 mm; Dicke/Stärke:: 15 mm; Höhe:: 3..."
